<a href="https://colab.research.google.com/github/FacuBelisario/Tesis-MSE/blob/master/SistemasDeControl/SistemaDeControlPID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
print("Probando el código")

Probando el código
